In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.compose import ColumnTransformer

In [2]:
dataset = pd.read_csv(r'C:\Users\BALA\Downloads\Bank_Churn.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\BALA\\Downloads\\Bank_Churn.csv'

In [ ]:
X = dataset.iloc[:, 3:13].values 
y = dataset.iloc[:, -1].values 

In [ ]:
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1, 2])], remainder='passthrough')
X_encoded = np.array(ct.fit_transform(X))

In [ ]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_encoded)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [ ]:
model = Sequential()


In [ ]:
model.add(Dense(units=6, activation='relu', input_dim=X_scaled.shape[1]))  # Input layer + first hidden layer


In [ ]:
model.add(Dense(units=1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, batch_size=10, epochs=100)

In [ ]:
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)


In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
sensitivity = np.sum((y_pred == 1) & (y_test == 1)) / np.sum(y_test == 1)
specificity = np.sum((y_pred == 0) & (y_test == 0)) / np.sum(y_test == 0)


In [ ]:
print("Loss:", loss)
print("Accuracy:", accuracy)
print("Sensitivity:", sensitivity)
print("Specificity:", specificity)


In [ ]:
y_pred_prob = model.predict(X_test)


# Lift Chart

In [ ]:
lift_chart = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred_prob.flatten()})


In [ ]:
lift_chart_sorted = lift_chart.sort_values(by='Predicted', ascending=False)


In [ ]:
lift_values = np.cumsum(lift_chart_sorted['Actual']) / np.sum(lift_chart_sorted['Actual'])


In [ ]:
lift_chart_sorted['Lift'] = lift_values


In [ ]:
lift_chart_sorted.reset_index(drop=True, inplace=True)


In [ ]:
print("Lift Chart:")
print(lift_chart_sorted)


# Calibration Chart

In [ ]:
calibration_chart = pd.DataFrame({'Predicted': y_pred_prob.flatten(), 'Actual': y_test})

In [ ]:
calibration_chart_sorted = calibration_chart.sort_values(by='Predicted')


In [ ]:
calibration_values = np.cumsum(calibration_chart_sorted['Actual']) / np.cumsum(np.ones(len(calibration_chart_sorted)))


In [ ]:
calibration_chart_sorted['Calibration'] = calibration_values


In [ ]:
calibration_chart_sorted.reset_index(drop=True, inplace=True)

In [ ]:
print("Calibration Chart:")
print(calibration_chart_sorted)

# Loss, Accuracy, Sensitivity, Specificity 

In [ ]:
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)

true_positives = np.sum((y_pred == 1) & (y_test == 1))
false_negatives = np.sum((y_pred == 0) & (y_test == 1))
true_negatives = np.sum((y_pred == 0) & (y_test == 0))
false_positives = np.sum((y_pred == 1) & (y_test == 0))
sensitivity = true_positives / (true_positives + false_negatives)
specificity = true_negatives / (true_negatives + false_positives)

print("Loss:", loss)
print("Accuracy:", accuracy)
print("Sensitivity:", sensitivity)
print("Specificity:", specificity)

# Number of Customers Leave/Stay

In [ ]:
new_dataset = pd.read_csv(r'C:\Users\BALA\Downloads\Bank_Churn.csv')

X_new = new_dataset.iloc[:, 3:13].values  

X_new_encoded = np.array(ct.transform(X_new))

X_new_scaled = scaler.transform(X_new_encoded)


churn_probabilities = model.predict(X_new_scaled)

threshold = 0.5

num_leave = np.sum(churn_probabilities > threshold)
num_stay = np.sum(churn_probabilities <= threshold)

print("Estimated number of customers likely to leave:", num_leave)
print("Estimated number of customers likely to stay:", num_stay)


# Highest Risk of Leaving

In [ ]:
highest_risk_indices = np.argsort(churn_probabilities.flatten())[-10:]
customers_at_highest_risk = dataset.iloc[highest_risk_indices].reset_index(drop=True)
print("Customers at the highest risk of leaving:")
print(customers_at_highest_risk)


In [ ]:
threshold = 0.5 

high_risk_customers = dataset[churn_probabilities > threshold]
print("Customers at high risk of leaving:")
print(high_risk_customers)